In [1]:
def is_done(path):
    amount_correct = 0
    first_date_time = ''
    last_date_time = ''
    sum_diff_data = 0.0
    with open(path, 'r') as fl:
        is_first_date_time = True
        for line in fl:
            is_done_ = re.match(r'100\%', line)
            if is_done_: 
                amount_correct += 1
    #print('amount correct submitions:', amount_correct)
    if amount_correct > 0: 
        done = 'yes'
        #print('done: yes')
    else: 
        done = 'no'
        #print('done: no')
    return amount_correct, done

In [2]:
def extract_subm_info(path):
    amount_correct, done = is_done(path)
    amount_submitions = 0
    first_date_time = ''
    last_date_time = ''
    sum_diff_data = 0.0
    submition_data_time = []
    with open(path, 'r') as file:
        is_first_date_time = True
        for line in file:
            submition_code = re.search(r'== SUBMITION \((.+)\)', line)
            if submition_code:
                
                #print('submition:', submition_code.group(1))
                submition_data_time.append(submition_code.group(1))
                amount_submitions += 1
                
                if is_first_date_time == True:
                    is_first_date_time = False
                    first_date_time = submition_code.group(1)#.replace('-', '/')
                    old_data = datetime.strptime(first_date_time, '%Y-%m-%d %H:%M:%S')
                
                last_date_time = submition_code.group(1)#.replace('-', '/')
                new_data = datetime.strptime(last_date_time, '%Y-%m-%d %H:%M:%S')
                if old_data <= new_data:
                    sum_diff_data += (new_data - old_data).seconds
                    old_data = new_data
                    
    #print('amount submitions:', amount_submitions)
    amount_incorrect = amount_submitions-amount_correct
    
    if first_date_time != '':
        first_date_time = datetime.strptime(first_date_time, '%Y-%m-%d %H:%M:%S')
        last_date_time = datetime.strptime(last_date_time, '%Y-%m-%d %H:%M:%S')
        data_diff = last_date_time - first_date_time
        variation_mean = sum_diff_data/amount_submitions
        variation_mean = round(variation_mean, 2)
        #print('first data submition try:', first_date_time)
        #print('last data submition try:', last_date_time)
        #print('variation data submition try:', data_diff.seconds)
        #print('mean of variations:', variation_mean)
        #values = [first_date_time.strftime('%Y-%m-%d %H:%M:%S'), 
        #         last_date_time.strftime('%Y-%m-%d %H:%M:%S'),
        values = [data_diff.seconds,
                  variation_mean, 
                  amount_submitions, 
                  amount_correct, 
                  amount_incorrect]
    else:
        #print('first data submition try:', 'NULL')
        #print('last data submition try:', 'NULL')
        #print('variation data submition try:', 'NULL')
        #print('mean of variations:', 'NULL')
        values = ['NULL','NULL', 
                  amount_submitions,
                  amount_correct,
                  amount_incorrect]
    return values, submition_data_time

In [3]:
def extract_test_info(path):
    amount_correct, done = is_done(path)
    amount_test = 0
    first_date_time = ''
    last_date_time = ''
    sum_diff_data = 0.0
    test_data_time = []
    with open(path, 'r') as file:
        is_first_date_time = True
        for line in file:
            test_code = re.search(r'== TEST \((.+)\)', line)
            if test_code:
                
                #print('test:', test_code.group(1))#.replace('-', '/'))
                test_data_time.append(test_code.group(1))
                amount_test += 1
                
                if is_first_date_time == True:
                    is_first_date_time = False
                    first_date_time = test_code.group(1)#.replace('-', '/')
                    old_data = datetime.strptime(first_date_time, '%Y-%m-%d %H:%M:%S')                
                
                last_date_time = test_code.group(1)#.replace('-', '/')
                new_data = datetime.strptime(last_date_time, '%Y-%m-%d %H:%M:%S')
                if old_data <= new_data:
                    sum_diff_data += (new_data - old_data).seconds
                    old_data = new_data
                                    
    #print('amount tests:',amount_test)
    
    if first_date_time != '':
        first_date_time = datetime.strptime(first_date_time, '%Y-%m-%d %H:%M:%S')
        last_date_time = datetime.strptime(last_date_time, '%Y-%m-%d %H:%M:%S')
        data_diff = last_date_time - first_date_time
        variation_mean = sum_diff_data/amount_test
        variation_mean = round(variation_mean, 2)
        #print('first data test try:', first_date_time)
       # print('last data test try:', last_date_time)
        #print('variation data test try:', data_diff.seconds)
        #print('mean of variations:', variation_mean)
        #values = [first_date_time.strftime('%Y-%m-%d %H:%M:%S'), 
        #          last_date_time.strftime('%Y-%m-%d %H:%M:%S'), 
        values = [data_diff.seconds,
                  variation_mean, 
                  amount_test, 
                  done]
    else:
        #print('first data test try:', 'NULL')
        #print('last data test try:', 'NULL')
        #print('variation data test try:', 'NULL')
        #print('mean of variations:', 'NULL')
        values = ['NULL','NULL', amount_test, done]
    return values, test_data_time


In [4]:
import os
import re
import pymysql

from pprint import pprint
from datetime import datetime

exercise_features = [
  'exercise_id',
  'user_id',
  'assessment_id',
  'general_variation_submit',
  'mean_variations_submit',
  'amount_submit',
  'amount_correct_submit',
  'amount_incorrect_submit',
  'general_variation_test',
  'mean_variations_test',
  'amount_test',
  'done',

]

def connection_database():
    connection = pymysql.connect(host='localhost',
                             user='root',
                             password='open',
                             db='dataset-workbench',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)
    return connection

conn = connection_database()
cursor = conn.cursor()

cont = 0

for semester_id in os.listdir('../dataset/workbench/'):
    path0 = os.path.join('../dataset/workbench/', semester_id)
    for class_id in os.listdir(path0):
            path1 = os.path.join(path0, class_id, 'users')
            for user_id in os.listdir(path1):
                    path2 = os.path.join(path1, user_id, 'executions')
                    for execution_id in os.listdir(path2):
                        
                        path5 = os.path.join(path2, execution_id)
                        test, tests_data_time = extract_test_info(path5)
                        submit0, submitions_data_time = extract_subm_info(path5)
                        
                        assessment_id = re.sub(r'\_\d+.log', '', execution_id)
                        exercise_id = re.sub(r'\d+\_', '', execution_id)
                        exercise_id = re.sub(r'.log', '', exercise_id)
                        fk_user = f'{semester_id}#{class_id}#{user_id}'
                        fk_assessment = f'{semester_id}#{class_id}#{assessment_id}'
                        
                        values = f"{exercise_id}, '{fk_user}', '{fk_assessment}', "
                        for value in (submit0+test):
                            if type(value) == str and value != 'NULL':
                                values += f"'{value}', "
                            else: values += f"{value}, "
                            
                        features = ''
                        for feature in exercise_features:
                            features += f'{feature}, '
                        
                        insert_exercise = f"INSERT INTO executions ({features[:-2]}) VALUES ({values[:-2]}) "
                        if cont < 1000:
                            print(insert_exercise)
                            cont += 1
                        cursor.execute(insert_exercise)
                        
                        for submition in submitions_data_time:
                            features = f"user_id, assessment_id, submition_datetime, exercise_id"
                            values = f"'{fk_user}', '{fk_assessment}', '{submition}', {exercise_id}"
                            insert_submition = f"INSERT INTO submition({features}) VALUES ({values})"
                            if cont < 1000:
                                print(' ', insert_submition)
                            cursor.execute(insert_submition)
                            
                        for test in tests_data_time:
                            features = f"user_id, assessment_id, test_datetime, exercise_id"
                            values = f"'{fk_user}', '{fk_assessment}', '{test}', {exercise_id}"
                            insert_test = f"INSERT INTO test({features}) VALUES ({values})"
                            if cont < 1000:
                                print(' ', insert_test)
                            cursor.execute(insert_test)
                        
conn.commit()
conn.close()

INSERT INTO executions (exercise_id, user_id, assessment_id, general_variation_submit, mean_variations_submit, amount_submit, amount_correct_submit, amount_incorrect_submit, general_variation_test, mean_variations_test, amount_test, done) VALUES (2508, '2018-2#208#3754', '2018-2#208#1450', 0, 0.0, 1, 1, 0, 0, 0.0, 1, 'yes') 
  INSERT INTO submition(user_id, assessment_id, submition_datetime, exercise_id) VALUES ('2018-2#208#3754', '2018-2#208#1450', '2018-10-04 13:19:55', 2508)
  INSERT INTO test(user_id, assessment_id, test_datetime, exercise_id) VALUES ('2018-2#208#3754', '2018-2#208#1450', '2018-10-04 13:19:46', 2508)
INSERT INTO executions (exercise_id, user_id, assessment_id, general_variation_submit, mean_variations_submit, amount_submit, amount_correct_submit, amount_incorrect_submit, general_variation_test, mean_variations_test, amount_test, done) VALUES (1280, '2018-2#208#3754', '2018-2#208#1521', NULL, NULL, 0, 0, 0, 43221, 1440.7, 30, 'no') 
  INSERT INTO test(user_id, asses

INSERT INTO executions (exercise_id, user_id, assessment_id, general_variation_submit, mean_variations_submit, amount_submit, amount_correct_submit, amount_incorrect_submit, general_variation_test, mean_variations_test, amount_test, done) VALUES (2602, '2018-2#208#3754', '2018-2#208#1521', 22476, 5619.0, 4, 1, 3, 22786, 1752.77, 13, 'yes') 
  INSERT INTO submition(user_id, assessment_id, submition_datetime, exercise_id) VALUES ('2018-2#208#3754', '2018-2#208#1521', '2018-11-27 09:56:39', 2602)
  INSERT INTO submition(user_id, assessment_id, submition_datetime, exercise_id) VALUES ('2018-2#208#3754', '2018-2#208#1521', '2018-11-28 16:06:36', 2602)
  INSERT INTO submition(user_id, assessment_id, submition_datetime, exercise_id) VALUES ('2018-2#208#3754', '2018-2#208#1521', '2018-11-28 16:07:58', 2602)
  INSERT INTO submition(user_id, assessment_id, submition_datetime, exercise_id) VALUES ('2018-2#208#3754', '2018-2#208#1521', '2018-11-28 16:11:15', 2602)
  INSERT INTO test(user_id, asses

INSERT INTO executions (exercise_id, user_id, assessment_id, general_variation_submit, mean_variations_submit, amount_submit, amount_correct_submit, amount_incorrect_submit, general_variation_test, mean_variations_test, amount_test, done) VALUES (2567, '2018-2#208#3754', '2018-2#208#1510', 0, 0.0, 1, 1, 0, 809, 404.5, 2, 'yes') 
  INSERT INTO submition(user_id, assessment_id, submition_datetime, exercise_id) VALUES ('2018-2#208#3754', '2018-2#208#1510', '2018-11-12 13:44:28', 2567)
  INSERT INTO test(user_id, assessment_id, test_datetime, exercise_id) VALUES ('2018-2#208#3754', '2018-2#208#1510', '2018-11-12 13:30:49', 2567)
  INSERT INTO test(user_id, assessment_id, test_datetime, exercise_id) VALUES ('2018-2#208#3754', '2018-2#208#1510', '2018-11-12 13:44:18', 2567)
INSERT INTO executions (exercise_id, user_id, assessment_id, general_variation_submit, mean_variations_submit, amount_submit, amount_correct_submit, amount_incorrect_submit, general_variation_test, mean_variations_test, a

  INSERT INTO submition(user_id, assessment_id, submition_datetime, exercise_id) VALUES ('2018-2#208#3754', '2018-2#208#1324', '2018-08-07 23:38:32', 1333)
  INSERT INTO submition(user_id, assessment_id, submition_datetime, exercise_id) VALUES ('2018-2#208#3754', '2018-2#208#1324', '2018-08-07 23:41:00', 1333)
  INSERT INTO submition(user_id, assessment_id, submition_datetime, exercise_id) VALUES ('2018-2#208#3754', '2018-2#208#1324', '2018-08-09 08:53:43', 1333)
  INSERT INTO test(user_id, assessment_id, test_datetime, exercise_id) VALUES ('2018-2#208#3754', '2018-2#208#1324', '2018-08-07 23:38:24', 1333)
  INSERT INTO test(user_id, assessment_id, test_datetime, exercise_id) VALUES ('2018-2#208#3754', '2018-2#208#1324', '2018-08-07 23:40:52', 1333)
  INSERT INTO test(user_id, assessment_id, test_datetime, exercise_id) VALUES ('2018-2#208#3754', '2018-2#208#1324', '2018-08-08 14:13:55', 1333)
  INSERT INTO test(user_id, assessment_id, test_datetime, exercise_id) VALUES ('2018-2#208#375

  INSERT INTO test(user_id, assessment_id, test_datetime, exercise_id) VALUES ('2018-2#208#3754', '2018-2#208#1415', '2018-09-23 10:40:40', 1101)
  INSERT INTO test(user_id, assessment_id, test_datetime, exercise_id) VALUES ('2018-2#208#3754', '2018-2#208#1415', '2018-09-23 10:41:41', 1101)
  INSERT INTO test(user_id, assessment_id, test_datetime, exercise_id) VALUES ('2018-2#208#3754', '2018-2#208#1415', '2018-09-23 10:42:29', 1101)
  INSERT INTO test(user_id, assessment_id, test_datetime, exercise_id) VALUES ('2018-2#208#3754', '2018-2#208#1415', '2018-09-23 10:43:00', 1101)
  INSERT INTO test(user_id, assessment_id, test_datetime, exercise_id) VALUES ('2018-2#208#3754', '2018-2#208#1415', '2018-09-23 10:43:41', 1101)
INSERT INTO executions (exercise_id, user_id, assessment_id, general_variation_submit, mean_variations_submit, amount_submit, amount_correct_submit, amount_incorrect_submit, general_variation_test, mean_variations_test, amount_test, done) VALUES (1418, '2018-2#208#3754'

...

INSERT INTO executions (exercise_id, user_id, assessment_id, general_variation_submit, mean_variations_submit, amount_submit, amount_correct_submit, amount_incorrect_submit, general_variation_test, mean_variations_test, amount_test, done) VALUES (2754, '2018-2#208#3745', '2018-2#208#1434', 0, 0.0, 1, 1, 0, 619, 41.27, 15, 'yes') 
  INSERT INTO submition(user_id, assessment_id, submition_datetime, exercise_id) VALUES ('2018-2#208#3745', '2018-2#208#1434', '2018-09-25 08:18:21', 2754)
  INSERT INTO test(user_id, assessment_id, test_datetime, exercise_id) VALUES ('2018-2#208#3745', '2018-2#208#1434', '2018-09-25 08:07:52', 2754)
  INSERT INTO test(user_id, assessment_id, test_datetime, exercise_id) VALUES ('2018-2#208#3745', '2018-2#208#1434', '2018-09-25 08:08:39', 2754)
  INSERT INTO test(user_id, assessment_id, test_datetime, exercise_id) VALUES ('2018-2#208#3745', '2018-2#208#1434', '2018-09-25 08:09:04', 2754)
  INSERT INTO test(user_id, assessment_id, test_datetime, exercise_id) VAL

  INSERT INTO test(user_id, assessment_id, test_datetime, exercise_id) VALUES ('2018-2#208#3745', '2018-2#208#1415', '2018-09-18 09:40:48', 1101)
  INSERT INTO test(user_id, assessment_id, test_datetime, exercise_id) VALUES ('2018-2#208#3745', '2018-2#208#1415', '2018-09-18 09:41:27', 1101)
  INSERT INTO test(user_id, assessment_id, test_datetime, exercise_id) VALUES ('2018-2#208#3745', '2018-2#208#1415', '2018-09-18 09:41:38', 1101)
  INSERT INTO test(user_id, assessment_id, test_datetime, exercise_id) VALUES ('2018-2#208#3745', '2018-2#208#1415', '2018-09-18 09:42:00', 1101)
  INSERT INTO test(user_id, assessment_id, test_datetime, exercise_id) VALUES ('2018-2#208#3745', '2018-2#208#1415', '2018-09-18 09:42:50', 1101)
  INSERT INTO test(user_id, assessment_id, test_datetime, exercise_id) VALUES ('2018-2#208#3745', '2018-2#208#1415', '2018-09-18 09:44:36', 1101)
  INSERT INTO test(user_id, assessment_id, test_datetime, exercise_id) VALUES ('2018-2#208#3745', '2018-2#208#1415', '2018-0

INSERT INTO executions (exercise_id, user_id, assessment_id, general_variation_submit, mean_variations_submit, amount_submit, amount_correct_submit, amount_incorrect_submit, general_variation_test, mean_variations_test, amount_test, done) VALUES (1105, '2018-2#208#3745', '2018-2#208#1415', 381, 95.25, 4, 1, 3, 272, 22.67, 12, 'yes') 
  INSERT INTO submition(user_id, assessment_id, submition_datetime, exercise_id) VALUES ('2018-2#208#3745', '2018-2#208#1415', '2018-09-18 09:19:17', 1105)
  INSERT INTO submition(user_id, assessment_id, submition_datetime, exercise_id) VALUES ('2018-2#208#3745', '2018-2#208#1415', '2018-09-18 09:20:23', 1105)
  INSERT INTO submition(user_id, assessment_id, submition_datetime, exercise_id) VALUES ('2018-2#208#3745', '2018-2#208#1415', '2018-09-18 09:21:24', 1105)
  INSERT INTO submition(user_id, assessment_id, submition_datetime, exercise_id) VALUES ('2018-2#208#3745', '2018-2#208#1415', '2018-09-18 09:25:38', 1105)
  INSERT INTO test(user_id, assessment_i

  INSERT INTO submition(user_id, assessment_id, submition_datetime, exercise_id) VALUES ('2018-2#208#3745', '2018-2#208#1480', '2018-10-25 10:02:00', 1580)
  INSERT INTO submition(user_id, assessment_id, submition_datetime, exercise_id) VALUES ('2018-2#208#3745', '2018-2#208#1480', '2018-10-29 13:53:43', 1580)
  INSERT INTO submition(user_id, assessment_id, submition_datetime, exercise_id) VALUES ('2018-2#208#3745', '2018-2#208#1480', '2018-10-29 13:55:23', 1580)
  INSERT INTO submition(user_id, assessment_id, submition_datetime, exercise_id) VALUES ('2018-2#208#3745', '2018-2#208#1480', '2018-10-29 13:55:51', 1580)
  INSERT INTO submition(user_id, assessment_id, submition_datetime, exercise_id) VALUES ('2018-2#208#3745', '2018-2#208#1480', '2018-10-29 15:49:28', 1580)
  INSERT INTO submition(user_id, assessment_id, submition_datetime, exercise_id) VALUES ('2018-2#208#3745', '2018-2#208#1480', '2018-10-29 15:50:23', 1580)
  INSERT INTO submition(user_id, assessment_id, submition_dateti

INSERT INTO executions (exercise_id, user_id, assessment_id, general_variation_submit, mean_variations_submit, amount_submit, amount_correct_submit, amount_incorrect_submit, general_variation_test, mean_variations_test, amount_test, done) VALUES (1803, '2018-2#208#3745', '2018-2#208#1348', 0, 0.0, 1, 1, 0, 620, 77.5, 8, 'yes') 
  INSERT INTO submition(user_id, assessment_id, submition_datetime, exercise_id) VALUES ('2018-2#208#3745', '2018-2#208#1348', '2018-08-21 09:20:26', 1803)
  INSERT INTO test(user_id, assessment_id, test_datetime, exercise_id) VALUES ('2018-2#208#3745', '2018-2#208#1348', '2018-08-21 09:09:58', 1803)
  INSERT INTO test(user_id, assessment_id, test_datetime, exercise_id) VALUES ('2018-2#208#3745', '2018-2#208#1348', '2018-08-21 09:10:35', 1803)
  INSERT INTO test(user_id, assessment_id, test_datetime, exercise_id) VALUES ('2018-2#208#3745', '2018-2#208#1348', '2018-08-21 09:12:07', 1803)
  INSERT INTO test(user_id, assessment_id, test_datetime, exercise_id) VALUE

In [5]:
print(str)

<class 'str'>
